In [1]:
import my_funcs as f

In [4]:
g_1105 = f.pd.read_csv(f.gdb+'geof/g1105')
#m_3022 = pd.read_csv(gdb+'geof/m3022.csv')

In [3]:
geof_1039 = pd.read_csv(gdb+'geof/geof_1039.csv')

In [10]:
g_1089 = pd.read_csv(gdb+'geof/g1089')
#m_1089 = pd.read_csv(gdb+'geof/m1089')

In [6]:
# Vetores com malha cartográfica em escala de 1:50.000
grids_1089_50k = f.gpd.read_file('/home/ggrl/Desktop/1089_area_50k.shp')
grids_1089_50k.head(10)

grids_1089_50k.plot('id_folha')

plt.figure(figsize=(9,9))
grids['eU'].eU.plot(cmap='hsv',vmin=0)
plt.axis('scaled')

plt.figure(figsize=(9,9))
grids['MAGR'].MAGR.plot(cmap='hsv')
plt.axis('scaled')

plt.figure(figsize=(9,9))
grids['CTC'].CTC.plot(cmap='hsv')
plt.axis('scaled')

plt.figure(figsize=(9,9))
grids['UC'].UC.plot(cmap='hsv')
plt.axis('scaled')

plt.figure(figsize=(9,9))
grids['THC'].THC.plot(cmap='hsv')
plt.axis('scaled')

plt.figure(figsize=(9,9))
grids['KC'].KC.plot(cmap='hsv',vmin=0)
plt.axis('scaled')

#
plt.figure()
plt.hist(scores['MDT'],bins='auto')

plt.figure()
plt.hist(scores['MAGR'],bins='auto')

plt.figure()
plt.hist(scores['CTC'],bins='auto')

plt.figure()
plt.hist(scores['THC'])

plt.figure()
plt.hist(scores['KC'])

plt.figure()
grid_uc.UC.plot(cmap='hsv')
plt.axis('scaled')

plt.figure()
plt.hist(scores_uc,bins='auto')

chain = vd.Chain([
        ('trend', vd.Trend(degree=3)),
        ('reduce', vd.BlockReduce(np.median, spacing=1000)),
        ('spline', vd.Spline())
    ])
    
chain.fit(coordinates, geof_SF23_Y_A_VI_3_DEM.UC)

cv     = vd.BlockKFold(spacing=1000,
                  n_splits=100,
                  shuffle=True)

scores_uc = vd.cross_val_score(chain,
                            coordinates,
                            geof_SF23_Y_A_VI_3_DEM.UC,
                            cv=cv)
    
grid_1 = chain.grid(spacing=200, data_names='UC',pixel_register=True)
grid_uc = vd.distance_mask(coordinates, maxdist=1000, grid= grid_1)

DriverError: /home/ggrl/Desktop/1089_area_50k.shp: No such file or directory

In [33]:
data = list()
for i in chain_list:
    df = grids[i].to_dataframe()
    data.append(df[i])

In [34]:
data = list()
for i in chain_list:
    df = grids[i].to_dataframe()
    data.append(df[i])

geof = pd.concat(data,axis=1, join='inner')

geof.reset_index(inplace=True)
geof['geometry'] = [geometry.Point(x,y) for x, y in zip(geof['easting'], geof['northing'])]

gdf = gpd.GeoDataFrame(geof,crs=32723)
gdf = gdf.set_crs(32723, allow_override=True)


geof['closest_unid'] = gdf['geometry'].apply(lambda x: socorro['SIGLA'].iloc[socorro.distance(x).idxmin()]) # .idxmin() Retorna o indice do menor valor 

,easting,northing,CTCOR,KPERC,MDT,eU,eth
0,500055.289543,8.784035e+06,1708.875338,1.685567,508.936948,1.907549,17.543268
1,500055.289543,8.784135e+06,1712.006845,1.627513,508.831103,1.922783,17.808410
2,500055.289543,8.784235e+06,1693.736960,1.561761,509.035380,1.935492,17.728627
3,500055.289543,8.784335e+06,1639.938707,1.493374,510.118943,1.950659,17.013406
4,500055.289543,8.784435e+06,1561.172489,1.426346,511.533855,1.962516,15.848140
...,...,...,...,...,...,...,...
302486,554643.870457,8.838854e+06,1138.891026,0.760344,471.539519,1.726946,12.759650
302487,554643.870457,8.838954e+06,1114.599345,0.783834,466.898206,1.429977,12.531083
302488,554643.870457,8.839054e+06,1088.677543,0.806869,462.777227,1.143662,12.318743
302489,554643.870457,8.839154e+06,1059.884453,0.828355,459.411416,0.881483,12.125568


In [40]:
geof['geometry'] = [geometry.Point(x,y) for x, y in zip(geof['easting'], geof['northing'])]

In [41]:
gdf = gpd.GeoDataFrame(geof,crs=32723)
gdf = gdf.set_crs(32723, allow_override=True)

gdf

,easting,northing,CTCOR,KPERC,MDT,eU,eth,geometry,MAGIGRF
0,500055.289543,8.784035e+06,1708.875338,1.685567,508.936948,1.907549,17.543268,POINT (500055.290 8784035.247),50.464271
1,500055.289543,8.784135e+06,1712.006845,1.627513,508.831103,1.922783,17.808410,POINT (500055.290 8784135.280),50.678373
2,500055.289543,8.784235e+06,1693.736960,1.561761,509.035380,1.935492,17.728627,POINT (500055.290 8784235.314),50.935766
3,500055.289543,8.784335e+06,1639.938707,1.493374,510.118943,1.950659,17.013406,POINT (500055.290 8784335.347),51.153968
4,500055.289543,8.784435e+06,1561.172489,1.426346,511.533855,1.962516,15.848140,POINT (500055.290 8784435.381),51.291826
...,...,...,...,...,...,...,...,...,...
302486,554643.870457,8.838854e+06,1138.891026,0.760344,471.539519,1.726946,12.759650,POINT (554643.870 8838853.619),37.771953
302487,554643.870457,8.838954e+06,1114.599345,0.783834,466.898206,1.429977,12.531083,POINT (554643.870 8838953.653),30.463146
302488,554643.870457,8.839054e+06,1088.677543,0.806869,462.777227,1.143662,12.318743,POINT (554643.870 8839053.686),24.237343
302489,554643.870457,8.839154e+06,1059.884453,0.828355,459.411416,0.881483,12.125568,POINT (554643.870 8839153.720),20.059724


In [26]:
df_mag = grids_mag.to_dataframe()

In [29]:
df_mag.reset_index(inplace=True)

In [30]:
df_mag

,easting,northing,MAGIGRF
0,500051.800009,8.784028e+06,50.464271
1,500051.800009,8.784128e+06,50.678373
2,500051.800009,8.784228e+06,50.935766
3,500051.800009,8.784328e+06,51.153968
4,500051.800009,8.784428e+06,51.291826
...,...,...,...
302486,554651.809991,8.838855e+06,37.771953
302487,554651.809991,8.838955e+06,30.463146
302488,554651.809991,8.839055e+06,24.237343
302489,554651.809991,8.839155e+06,20.059724


In [38]:
geof['MAGIGRF'] = df_mag.MAGIGRF

In [39]:
geof

,easting,northing,CTCOR,KPERC,MDT,eU,eth,geometry,MAGIGRF
0,500055.289543,8.784035e+06,1708.875338,1.685567,508.936948,1.907549,17.543268,POINT (500055.290 8784035.247),50.464271
1,500055.289543,8.784135e+06,1712.006845,1.627513,508.831103,1.922783,17.808410,POINT (500055.290 8784135.280),50.678373
2,500055.289543,8.784235e+06,1693.736960,1.561761,509.035380,1.935492,17.728627,POINT (500055.290 8784235.314),50.935766
3,500055.289543,8.784335e+06,1639.938707,1.493374,510.118943,1.950659,17.013406,POINT (500055.290 8784335.347),51.153968
4,500055.289543,8.784435e+06,1561.172489,1.426346,511.533855,1.962516,15.848140,POINT (500055.290 8784435.381),51.291826
...,...,...,...,...,...,...,...,...,...
302486,554643.870457,8.838854e+06,1138.891026,0.760344,471.539519,1.726946,12.759650,POINT (554643.870 8838853.619),37.771953
302487,554643.870457,8.838954e+06,1114.599345,0.783834,466.898206,1.429977,12.531083,POINT (554643.870 8838953.653),30.463146
302488,554643.870457,8.839054e+06,1088.677543,0.806869,462.777227,1.143662,12.318743,POINT (554643.870 8839053.686),24.237343
302489,554643.870457,8.839154e+06,1059.884453,0.828355,459.411416,0.881483,12.125568,POINT (554643.870 8839153.720),20.059724


In [137]:
gdf.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [42]:
l_100k = mf.litologia('l_100k')

In [43]:
rparaim = mf.mapa(l_100k,'Rio Paraim')

In [44]:
rparaim.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [45]:
rparaim = rparaim.to_crs(32723)
rparaim.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [49]:
rparaim.reset_index(inplace=True)

In [50]:
rparaim

,index,ID_UNIDADE,SIGLA,HIERARQUIA,NOME,AMBIENTE_T,SUB_AMBIEN,SIGLA_PAI,NOME_PAI,LEGENDA,...,ERA_MIN,ERA_MAX,SISTEMA_MI,SISTEMA_MA,EPOCA_MIN,EPOCA_MAX,SIGLAS_HIS,SHAPE_AREA,SHAPE_LEN,geometry
0,2816,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.000041,0.037400,"POLYGON ((500176.894 8821390.751, 500170.654 8..."
1,2820,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.001087,0.257896,"POLYGON ((505306.762 8803062.104, 505258.182 8..."
2,2821,4873,NP1rp3,Unidade,"Unidade Rio Preto, xisto",None,None,NP1rp,NP1rp - Grupo Rio Preto,Muscovita-xistos e quartzo-xistos predominante...,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"MP3NP1rp3, M3N1rp1x, MP3NP1rp1x",0.000170,0.094417,"POLYGON ((505145.795 8801235.292, 506323.529 8..."
3,2823,4876,NP1rp2q,Litofácies,"Litofácies Rio Preto, metassedimentos quartzos...",None,None,MP3NP1rp2,"MP3NP1rp2 - Unidade Rio Preto, metassedimentos...",Quartzitos a muscovita-quartzitos puros a impu...,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"M3N1rp2q, MP3NP1rp2q",0.000015,0.020423,"POLYGON ((509741.424 8784951.753, 509758.753 8..."
4,2825,5773,PP2cmv,Unidade,"Unidade Cristalândia do Piauí, metamáfica",None,None,A4PPc,A4PPc - Complexo Cristalândia do Piauí,"Corpos gabróicos a metagabros, compostos por a...",...,Paleoproterozóico,Paleoproterozóico,Riaciano,Riaciano,Série 3,Série 3,None,0.000323,0.077072,"POLYGON ((510857.857 8811448.300, 510816.261 8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,54750,4876,NP1rp2q,Litofácies,"Litofácies Rio Preto, metassedimentos quartzos...",None,None,MP3NP1rp2,"MP3NP1rp2 - Unidade Rio Preto, metassedimentos...",Quartzitos a muscovita-quartzitos puros a impu...,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"M3N1rp2q, MP3NP1rp2q",0.000045,0.063783,"POLYGON ((548787.138 8829535.408, 548759.773 8..."
137,54752,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.012576,0.986684,"POLYGON ((543318.427 8837536.690, 543374.133 8..."
138,54754,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.043917,1.961410,"POLYGON ((554660.641 8807055.364, 554656.953 8..."
139,54755,4873,NP1rp3,Unidade,"Unidade Rio Preto, xisto",None,None,NP1rp,NP1rp - Grupo Rio Preto,Muscovita-xistos e quartzo-xistos predominante...,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"MP3NP1rp3, M3N1rp1x, MP3NP1rp1x",0.023544,3.651426,"POLYGON ((554702.380 8832876.999, 554702.379 8..."


In [46]:
rparaim.plot('SIGLA')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [51]:
geof['closest_unid'] = gdf['geometry'].apply(lambda x: rparaim['SIGLA'].iloc[rparaim.distance(x).idxmin()]) # .idxmin() Retorna o indice do menor valor 

In [52]:
geof.head()

,easting,northing,CTCOR,KPERC,MDT,eU,eth,geometry,MAGIGRF,closest_unid
0,500055.289543,8.784035e+06,1708.875338,1.685567,508.936948,1.907549,17.543268,POINT (500055.290 8784035.247),50.464271,NP1rp3
1,500055.289543,8.784135e+06,1712.006845,1.627513,508.831103,1.922783,17.808410,POINT (500055.290 8784135.280),50.678373,NP1rp3
2,500055.289543,8.784235e+06,1693.736960,1.561761,509.035380,1.935492,17.728627,POINT (500055.290 8784235.314),50.935766,NP1rp3
3,500055.289543,8.784335e+06,1639.938707,1.493374,510.118943,1.950659,17.013406,POINT (500055.290 8784335.347),51.153968,NP1rp3
4,500055.289543,8.784435e+06,1561.172489,1.426346,511.533855,1.962516,15.848140,POINT (500055.290 8784435.381),51.291826,NP1rp3


In [53]:
# Setting projections

# South America WGS 84 Geográfica
p1 = pyproj.Proj("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

# WGS84 UTM 
p2 = pyproj.Proj("+proj=utm +zone=23 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

# 1105
geof['LONGITUDE'], geof['LATITUDE'] = pyproj.transform(p2, p1,
                                                geof.easting.values,
                                                geof.northing.values)

<ipython-input-53-229dbd804a89>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  geof['LONGITUDE'], geof['LATITUDE'] = pyproj.transform(p2, p1,


In [54]:
geof.head()

,easting,northing,CTCOR,KPERC,MDT,eU,eth,geometry,MAGIGRF,closest_unid,LONGITUDE,LATITUDE
0,500055.289543,8.784035e+06,1708.875338,1.685567,508.936948,1.907549,17.543268,POINT (500055.290 8784035.247),50.464271,NP1rp3,-44.999494,-10.999867
1,500055.289543,8.784135e+06,1712.006845,1.627513,508.831103,1.922783,17.808410,POINT (500055.290 8784135.280),50.678373,NP1rp3,-44.999494,-10.998963
2,500055.289543,8.784235e+06,1693.736960,1.561761,509.035380,1.935492,17.728627,POINT (500055.290 8784235.314),50.935766,NP1rp3,-44.999494,-10.998058
3,500055.289543,8.784335e+06,1639.938707,1.493374,510.118943,1.950659,17.013406,POINT (500055.290 8784335.347),51.153968,NP1rp3,-44.999494,-10.997153
4,500055.289543,8.784435e+06,1561.172489,1.426346,511.533855,1.962516,15.848140,POINT (500055.290 8784435.381),51.291826,NP1rp3,-44.999494,-10.996248


In [55]:
geof.to_csv(gdb+'OI_SC23_Z_C_I.csv',index=False)